In [1]:
# QUESTION
#
# For this task you will need to train a neural network
# to predict sunspot activity using the Sunspots.csv dataset.
# Your neural network must  have an MAE
# of 0.12 or less on the normalized dataset for top marks.
# Code for normalizing the data is provided and should not be changed.
# At the bottom of this file, we provide  some testing
# code should you want to check your model.

# Note: Do not use lambda layers in your model, they are not supported
# on the grading infrastructure.
# <번역> 
# 이 작업을 위해서는 신경망을 훈련시켜야합니다.
# Sunspots.csv 데이터 세트를 사용하여 흑점 활동을 예측합니다.
# 상위 마크에 대한 정규화 된 데이터 세트에서 0.12 이하의 MAE가 있어야합니다.
# 데이터 정규화를위한 코드가 제공되며 변경해서는 안됩니다.
# 당신의 모델을 체크하려면, 이 파일의 맨 아래에 몇 가지 테스팅 코드가 있습니다.
# 참고 : 모델에서 람다 레이어를 사용하지 마십시오. 채점 인프라에서 지원되지 않습니다.
 


import csv
import tensorflow as tf
import numpy as np
import urllib
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, LSTM 
from tensorflow.keras.optimizers import SGD, RMSprop
from tensorflow.keras.losses import Huber
from tensorflow.keras.callbacks import ModelCheckpoint


# DO NOT CHANGE THIS CODE
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)


def solution_model():
    url = 'https://storage.googleapis.com/download.tensorflow.org/data/Sunspots.csv'
    urllib.request.urlretrieve(url, 'sunspots.csv')

    time_step = []
    sunspots = []

    with open('sunspots.csv') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        next(reader)
        for row in reader:
            sunspots.append(float(row[2]))   # YOUR CODE HERE
            time_step.append(int(row[0]))  # YOUR CODE HERE

    series =  np.array(sunspots)# YOUR CODE HERE

    # DO NOT CHANGE THIS CODE
    # This is the normalization function
    min = np.min(series)
    max = np.max(series)
    series -= min
    series /= max
    time = np.array(time_step)

    # The data should be split into training and validation sets at time step 3000
    # DO NOT CHANGE THIS CODE
    split_time = 3000


    time_train = time[:split_time]# YOUR CODE HERE
    x_train = series[:split_time]# YOUR CODE HERE
    time_valid = time[split_time:]# YOUR CODE HERE
    x_valid = series[split_time:] # YOUR CODE HERE

    # DO NOT CHANGE THIS CODE
    window_size = 30
    batch_size = 32
    shuffle_buffer_size = 1000

    train_set = windowed_dataset(x_train, window_size=window_size, batch_size=batch_size, shuffle_buffer=shuffle_buffer_size)
    valid_set = windowed_dataset(x_valid, window_size=window_size, batch_size=batch_size, shuffle_buffer=shuffle_buffer_size)

    model = tf.keras.models.Sequential([
        Conv1D(60, kernel_size=5, padding='causal', activation='relu', input_shape=[None,1]),
        LSTM(60,return_sequences=True),
        LSTM(60, return_sequences=True),
        Dense(30, activation='relu'),
        Dense(10, activation='relu'),                                  
        # YOUR CODE HERE. Whatever your first layer is, the input shape will be [None,1] when using the Windowed_dataset above, depending on the layer type chosen
        tf.keras.layers.Dense(1)
    ])
    opt = SGD(lr=1e-5, momentum=0.9)
    model.compile(loss=Huber(), optimizer=opt, metrics=['mae'])
    
    checkpoint_path = 'sunset_checkpoint1.ckpt'
    checkpoint = ModelCheckpoint(filepath = checkpoint_path, save_weights_only=True, save_best_only=True, 
                                monitor='val_mae', verbose=1)
    model.fit(train_set, validation_data = (valid_set), epochs=100, callbacks=[checkpoint])
    model.load_weights(checkpoint_path)
    # YOUR CODE HERE TO COMPILE AND TRAIN THE MODEL
    return model


# Note that you'll need to save your model as a .h5 like this.
# When you press the Submit and Test button, this .h5 model will be
# sent to the testing infrastructure for scoring.

# You must use the Submit and Test button to submit your model
# at least once in each category before you finally submit your exam.

if __name__ == '__main__':
    model = solution_model()
    model.save("cat5_sunspots(typeB).h5")



# THIS CODE IS USED IN THE TESTER FOR FORECASTING. IF YOU WANT TO TEST YOUR MODEL
# BEFORE UPLOADING YOU CAN DO IT WITH THIS
#def model_forecast(model, series, window_size):
#    ds = tf.data.Dataset.from_tensor_slices(series)
#    ds = ds.window(window_size, shift=1, drop_remainder=True)
#    ds = ds.flat_map(lambda w: w.batch(window_size))
#    ds = ds.batch(32).prefetch(1)
#    forecast = model.predict(ds)
#    return forecast


#window_size = # YOUR CODE HERE
#rnn_forecast = model_forecast(model, series[..., np.newaxis], window_size)
#rnn_forecast = rnn_forecast[split_time - window_size:-1, -1, 0]

#result = tf.keras.metrics.mean_absolute_error(x_valid, rnn_forecast).numpy()

## To get the maximum score, your model must have an MAE OF .12 or less.
## When you Submit and Test your model, the grading infrastructure
## converts the MAE of your model to a score from 0 to 5 as follows:

#test_val = 100 * result
#score = math.ceil(17 - test_val)
#if score > 5:
#    score = 5

#print(score)

Epoch 1/100
     92/Unknown - 1s 14ms/step - loss: 0.0320 - mae: 0.1923
Epoch 00001: val_mae improved from inf to 0.16108, saving model to sunset_checkpoint1.ckpt
93/93 [==============================] - 2s 25ms/step - loss: 0.0321 - mae: 0.1926 - val_loss: 0.0219 - val_mae: 0.1611
Epoch 2/100
93/93 [==============================] - ETA: 0s - loss: 0.0305 - mae: 0.1857
Epoch 00002: val_mae improved from 0.16108 to 0.15523, saving model to sunset_checkpoint1.ckpt
93/93 [==============================] - 1s 16ms/step - loss: 0.0305 - mae: 0.1857 - val_loss: 0.0208 - val_mae: 0.1552
Epoch 3/100
92/93 [============================>.] - ETA: 0s - loss: 0.0293 - mae: 0.1802
Epoch 00003: val_mae improved from 0.15523 to 0.15014, saving model to sunset_checkpoint1.ckpt
93/93 [==============================] - 1s 16ms/step - loss: 0.0292 - mae: 0.1799 - val_loss: 0.0197 - val_mae: 0.1501
Epoch 4/100
93/93 [==============================] - ETA: 0s - loss: 0.0280 - mae: 0.1747
Epoch 00004: val_

Epoch 28/100
92/93 [============================>.] - ETA: 0s - loss: 0.0153 - mae: 0.1285
Epoch 00028: val_mae improved from 0.10697 to 0.10660, saving model to sunset_checkpoint1.ckpt
93/93 [==============================] - 1s 15ms/step - loss: 0.0153 - mae: 0.1286 - val_loss: 0.0096 - val_mae: 0.1066
Epoch 29/100
90/93 [============================>.] - ETA: 0s - loss: 0.0150 - mae: 0.1274
Epoch 00029: val_mae improved from 0.10660 to 0.10630, saving model to sunset_checkpoint1.ckpt
93/93 [==============================] - 1s 15ms/step - loss: 0.0152 - mae: 0.1281 - val_loss: 0.0095 - val_mae: 0.1063
Epoch 30/100
93/93 [==============================] - ETA: 0s - loss: 0.0150 - mae: 0.1276
Epoch 00030: val_mae improved from 0.10630 to 0.10605, saving model to sunset_checkpoint1.ckpt
93/93 [==============================] - 1s 15ms/step - loss: 0.0150 - mae: 0.1276 - val_loss: 0.0094 - val_mae: 0.1060
Epoch 31/100
92/93 [============================>.] - ETA: 0s - loss: 0.0148 - mae

Epoch 58/100
93/93 [==============================] - ETA: 0s - loss: 0.0128 - mae: 0.1240
Epoch 00058: val_mae did not improve from 0.10528
93/93 [==============================] - 1s 14ms/step - loss: 0.0128 - mae: 0.1240 - val_loss: 0.0084 - val_mae: 0.1065
Epoch 59/100
90/93 [============================>.] - ETA: 0s - loss: 0.0127 - mae: 0.1238
Epoch 00059: val_mae did not improve from 0.10528
93/93 [==============================] - 1s 14ms/step - loss: 0.0127 - mae: 0.1240 - val_loss: 0.0084 - val_mae: 0.1065
Epoch 60/100
89/93 [===========================>..] - ETA: 0s - loss: 0.0126 - mae: 0.1234
Epoch 00060: val_mae did not improve from 0.10528
93/93 [==============================] - 1s 14ms/step - loss: 0.0127 - mae: 0.1240 - val_loss: 0.0084 - val_mae: 0.1066
Epoch 61/100
93/93 [==============================] - ETA: 0s - loss: 0.0127 - mae: 0.1239
Epoch 00061: val_mae did not improve from 0.10528
93/93 [==============================] - 1s 14ms/step - loss: 0.0127 - mae: 

Epoch 90/100
91/93 [============================>.] - ETA: 0s - loss: 0.0121 - mae: 0.1232
Epoch 00090: val_mae did not improve from 0.10528
93/93 [==============================] - 1s 11ms/step - loss: 0.0122 - mae: 0.1236 - val_loss: 0.0084 - val_mae: 0.1077
Epoch 91/100
91/93 [============================>.] - ETA: 0s - loss: 0.0121 - mae: 0.1234
Epoch 00091: val_mae did not improve from 0.10528
93/93 [==============================] - 1s 11ms/step - loss: 0.0122 - mae: 0.1236 - val_loss: 0.0084 - val_mae: 0.1077
Epoch 92/100
91/93 [============================>.] - ETA: 0s - loss: 0.0122 - mae: 0.1235
Epoch 00092: val_mae did not improve from 0.10528
93/93 [==============================] - 1s 11ms/step - loss: 0.0122 - mae: 0.1236 - val_loss: 0.0084 - val_mae: 0.1077
Epoch 93/100
91/93 [============================>.] - ETA: 0s - loss: 0.0121 - mae: 0.1235
Epoch 00093: val_mae did not improve from 0.10528
93/93 [==============================] - 1s 11ms/step - loss: 0.0122 - mae: 

In [3]:
# <참고용코드-모델의 성능을 테스트하기 위한 코드 5등급 ok> #
# 이부분은 절대 답으로 포함시키지 마세요 #
# THIS CODE IS USED IN THE TESTER FOR FORECASTING. IF YOU WANT TO TEST YOUR MODEL
# BEFORE UPLOADING YOU CAN DO IT WITH THIS
import math
url = 'https://storage.googleapis.com/download.tensorflow.org/data/Sunspots.csv'
urllib.request.urlretrieve(url, 'sunspots.csv')

time_step = []
sunspots = []

with open('sunspots.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
        sunspots.append(float(row[2]))   # YOUR CODE HERE
        time_step.append(int(row[0]))  # YOUR CODE HERE

series =  np.array(sunspots)# YOUR CODE HERE

# DO NOT CHANGE THIS CODE
# This is the normalization function
min = np.min(series)
max = np.max(series)
series -= min
series /= max
time = np.array(time_step)
split_time = 3000
time_train = time[:split_time]# YOUR CODE HERE
x_train = series[:split_time]# YOUR CODE HERE
time_valid = time[split_time:]# YOUR CODE HERE
x_valid = series[split_time:] # YOUR CODE HERE

# DO NOT CHANGE THIS CODE
window_size = 30
batch_size = 32
shuffle_buffer_size = 1000
    
def model_forecast(model, series, window_size):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size))
    ds = ds.batch(32).prefetch(1)
    forecast = model.predict(ds)
    return forecast

# window_size =  30# YOUR CODE HERE
rnn_forecast = model_forecast(model, series[..., np.newaxis], window_size)
rnn_forecast = rnn_forecast[split_time - window_size:-1, -1, 0]

result = tf.keras.metrics.mean_absolute_error(x_valid, rnn_forecast).numpy()

# To get the maximum score, your model must have an MAE OF .12 or less.
# When you Submit and Test your model, the grading infrastructure
# converts the MAE of your model to a score from 0 to 5 as follows:

test_val = 100 * result
score = math.ceil(17 - test_val)
if score > 5:
    score = 5

print(score)

5
